In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [2]:
class ChargeEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Peice array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start SOC
        self.state = 20 #+ random.randint(-3,3)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 sell
        # 1 -1 = 0 hold
        # 2 -1 = 1 buy 
        self.state += self.change * self.power  

        
        
        if self.state == 100:
            action = 1
        if self.state == 0:
            action = 0
            
        # Calculate reward    
        # BUy
        # Give plus reward is charge at low proce, and minus at high price
        if action == 0:
            self.change = 1
            #if (self.max_state - self.state) < self.power:
                #self.temp = self.max_state - self.state
                #self.state = 100
            if self.priceArray[self.currentIndex] < self.mid - 2:
                reward = self.priceArray[self.currentIndex]
            else:
                reward = -1 * self.priceArray[self.currentIndex]
                    
        # Sell
        elif action == 1:
                #if self.state == 100 or self.priceArray[self.currentIndex] > 11 : 
                #    if self.state < self.power :
                #        self.temp = self.state - 0
                #        reward = 1 * self.priceArray[self.currentIndex] * (self.temp/self.power)
                #        self.state = 0
                #    else:
                #        reward = 1 * self.priceArray[self.currentIndex]
                #else:
                #    self.change = 0
                #    reward = -1
            self.change = -1
            #if self.state < self.power :
                #self.temp = self.state - 0
                #self.state = 0
            if self.priceArray[self.currentIndex] > self.mid + 2:
                reward = 1 * self.priceArray[self.currentIndex]
            else:
                reward = -1 * self.priceArray[self.currentIndex]
                
        # Hold        
        else:
            self.change = 0
            if self.priceArray[self.currentIndex] > self.mid + 2 and self.priceArray[self.currentIndex] < self.mid - 2:
                reward = 1
            else:
                reward = -5
        
        # Move to the next price
        self.currentIndex += 1
        
        # Set Low boundary and Up boundary
        #Set end state
        if self.currentIndex == len(self.priceArray)-1: 
            done = True
            if self.state == 100:
                reward += 50
            elif self.state >= 90:
                reward += 30
            elif self.state >= 80:
                reward += 10
                    
            else:
                reward = -100
        elif self.state < 0 or self.state > 100:
                done = True
                reward = -100
        else:
            done = False       

        

        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = 20 #+ random.randint(-3,3)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 14.18,
                                               16.11, 21.42, 25.87, 30.71, 18.44]))
        # Set current index
        self.currentIndex = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        # Set median
        self.mid = np.median(self.priceArray)
        return self.state

In [3]:
env = ChargeEnv()

C:\Users\Yuche\AppData\Roaming\Python\Python39\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
env.observation_space.sample()

array([79.18931], dtype=float32)

In [5]:
episodes = 100
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))

Episode:1 Score:-191.72999912500381 State:20
Episode:2 Score:-91.58000254631042 State:60
Episode:3 Score:-75.86000162363052 State:20
Episode:4 Score:-176.46000236272812 State:40
Episode:5 Score:-223.25000166893005 State:60
Episode:6 Score:-159.53999775648117 State:0
Episode:7 Score:-105.49000066518784 State:40
Episode:8 Score:-180.31000238656998 State:40
Episode:9 Score:-160.30999940633774 State:40
Episode:10 Score:-206.0600009560585 State:60
Episode:11 Score:-190.05000162124634 State:80
Episode:12 Score:-101.32000279426575 State:80
Episode:13 Score:-162.5199996829033 State:0
Episode:14 Score:-182.63000172376633 State:40
Episode:15 Score:-141.6899978518486 State:40
Episode:16 Score:-173.79999989271164 State:40
Episode:17 Score:-152.3299970626831 State:60
Episode:18 Score:-92.60999727249146 State:80
Episode:19 Score:-129.41000074148178 State:20
Episode:20 Score:-176.9099993109703 State:0
Episode:21 Score:-332.65000051259995 State:60
Episode:22 Score:-136.62000107765198 State:60
Episode:

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [7]:
states = env.observation_space.shape
actions = env.action_space.n

In [8]:
actions

3

In [9]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [16]:
# Apply when 'Sequential' object has no attribute '_compile_time_distribution_strategy'
del model 

In [17]:
model = build_model(states, actions)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 24)                48        
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [19]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import gym

In [20]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [21]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4), metrics=['mse']) #mean squared error mse, mean absloute error mae.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)


C:\Users\Yuche\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2352: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  560/10000 [>.............................] - ETA: 1:09 - reward: -0.4553done, took 5.463 seconds


In [22]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 0.130, steps: 23
Episode 2: reward: 0.130, steps: 23
Episode 3: reward: 0.130, steps: 23
Episode 4: reward: 0.130, steps: 23
Episode 5: reward: 0.130, steps: 23
Episode 6: reward: 0.130, steps: 23
Episode 7: reward: 0.130, steps: 23
Episode 8: reward: 0.130, steps: 23
Episode 9: reward: 0.130, steps: 23
Episode 10: reward: 0.130, steps: 23
Episode 11: reward: 0.130, steps: 23
Episode 12: reward: 0.130, steps: 23
Episode 13: reward: 0.130, steps: 23
Episode 14: reward: 0.130, steps: 23
Episode 15: reward: 0.130, steps: 23
Episode 16: reward: 0.130, steps: 23
Episode 17: reward: 0.130, steps: 23
Episode 18: reward: 0.130, steps: 23
Episode 19: reward: 0.130, steps: 23
Episode 20: reward: 0.130, steps: 23
Episode 21: reward: 0.130, steps: 23
Episode 22: reward: 0.130, steps: 23
Episode 23: reward: 0.130, steps: 23
Episode 24: reward: 0.130, steps: 23
Episode 25: reward: 0.130, steps: 23
Episode 26: reward: 0.130, steps: 23
Episode 27: reward

In [ ]:
#dqn.save_weights('dqn_weights.h5f', overwrite=True)